# Build your own NVIDIA Agent

The `llama-index-agent-nvidia` package contains LlamaIndex integrations building applications with models on 
NVIDIA NIM inference microservice. NVIDIA llama-3.1 endpoints supports function calling, it's never been easier to build your own agent!


NVIDIA hosted deployments of NIMs are available to test on the [NVIDIA API catalog](https://build.nvidia.com/). After testing, 
NIMs can be exported from NVIDIA’s API catalog using the NVIDIA AI Enterprise license and run on-premises or in the cloud, 
giving enterprises ownership and full control of their IP and AI application.

NIMs are packaged as container images on a per model basis and are distributed as NGC container images through the NVIDIA NGC Catalog. 
At their core, NIMs provide easy, consistent, and familiar APIs for running inference on an AI model.


In this notebook tutorial, we showcase how to write your own NVIDIA agent.

## Initial Setup 

Let's start by importing some simple building blocks.  

The main thing we need is:
1. the NVIDIA NIM Endpoint (using our own `llama_index` LLM class)
2. a place to keep conversation history 
3. a definition for tools that our agent can use.

In [ ]:
%pip install --upgrade --quiet llama-index-llms-nvidia llama-index-agent-nvidia


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import getpass
import os

# del os.environ['NVIDIA_API_KEY']  ## delete key and reset
if os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    print("Valid NVIDIA_API_KEY already in environment. Delete to reset")
else:
    nvapi_key = getpass.getpass("NVAPI Key (starts with nvapi-): ")
    assert nvapi_key.startswith(
        "nvapi-"
    ), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key

In [ ]:
import json
from typing import Sequence, List

from llama_index.llms.nvidia import NVIDIA
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from openai.types.chat import ChatCompletionMessageToolCall

Let's define some very simple calculator tools for our agent.

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [ ]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

## Agent Definition

Now, we define our agent that's capable of holding a conversation and calling tools in **under 50 lines of code**.

The meat of the agent logic is in the `chat` method. At a high-level, there are 3 steps:
1. Call NVIDIA LLM to decide which tool (if any) to call and with what arguments.
2. Call the tool with the arguments to obtain an output
3. Call NVIDIA LLM to synthesize a response from the conversation context and the tool output.

The `reset` method simply resets the conversation context, so we can start another conversation.

In [ ]:
class YourNVIDIAAgent:
    def __init__(
        self,
        tools: Sequence[BaseTool] = [],
        llm: NVIDIA = NVIDIA(
            temperature=0, model="meta/llama-3.1-8b-instruct"
        ),
        chat_history: List[ChatMessage] = [],
    ) -> None:
        self._llm = llm
        self._tools = {tool.metadata.name: tool for tool in tools}
        self._chat_history = chat_history

    def reset(self) -> None:
        self._chat_history = []

    def chat(self, message: str) -> str:
        chat_history = self._chat_history
        chat_history.append(ChatMessage(role="user", content=message))
        tools = [
            tool.metadata.to_openai_tool() for _, tool in self._tools.items()
        ]

        ai_message = self._llm.chat(chat_history, tools=tools).message
        additional_kwargs = ai_message.additional_kwargs
        chat_history.append(ai_message)

        tool_calls = additional_kwargs.get("tool_calls", None)
        # parallel function calling is now supported
        if tool_calls is not None:
            for tool_call in tool_calls:
                function_message = self._call_function(tool_call)
                chat_history.append(function_message)
                ai_message = self._llm.chat(chat_history).message
                chat_history.append(ai_message)

        return ai_message.content

    def _call_function(
        self, tool_call: ChatCompletionMessageToolCall
    ) -> ChatMessage:
        id_ = tool_call.id
        function_call = tool_call.function
        tool = self._tools[function_call.name]
        output = tool(**json.loads(function_call.arguments))
        return ChatMessage(
            name=function_call.name,
            content=str(output),
            role="tool",
            additional_kwargs={
                "tool_call_id": id_,
                "name": function_call.name,
            },
        )

In [ ]:
agent = YourNVIDIAAgent(tools=[multiply_tool, add_tool])

In [ ]:
agent.chat("Hi")

"It's nice to meet you. How can I assist you today?"

In [ ]:
agent.chat("What is 2123 * 215123")

'The result of 2123 * 215123 is 456706129.'

## `NVIDIAAgent` Implementation 

We provide a (slightly better) `NVIDIAAgent` implementation in LlamaIndex, which you can directly use as follows.  

In comparison to the simplified version above:
* it implements the `BaseChatEngine` and `BaseQueryEngine` interface, so you can more seamlessly use it in the LlamaIndex framework. 
* it supports multiple function calls per conversation turn
* it supports streaming
* it supports async endpoints
* it supports callback and tracing

In [ ]:
from llama_index.agent.nvidia import NVIDIAAgent

In [ ]:
agent = NVIDIAAgent.from_tools(
    [multiply_tool, add_tool],
    llm=NVIDIA("meta/llama-3.1-70b-instruct"),
    verbose=True,
)

### Chat

In [ ]:
response = agent.chat("What is (121 * 3) + 42?")
print(str(response))

Added user message to memory: What is (121 * 3) + 42?
=== Calling Function ===
Calling function: multiply with args: {"a": 121, "b": 3}
Got output: 363

=== Calling Function ===
Calling function: add with args: {"a": 363, "b": 42}
Got output: 405

The answer is 405.


In [ ]:
# inspect sources
print(response.sources)

[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363, is_error=False), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405, is_error=False)]


### Async Chat

In [ ]:
response = await agent.achat("What is 121 * 3?")
print(str(response))

Added user message to memory: What is 121 * 3?
=== Calling Function ===
Calling function: multiply with args: {"a": 121, "b": 3}
Got output: 363

The answer is 363.


### Streaming Chat
Here, every LLM response is returned as a generator. You can stream every incremental step, or only the last response.

In [ ]:
response = agent.stream_chat(
    "What is 121 * 2? Once you have the answer, use that number to write a"
    " story about a group of mice."
)

response_gen = response.response_gen

for token in response_gen:
    print(token, end="")

Added user message to memory: What is 121 * 2? Once you have the answer, use that number to write a story about a group of mice.
=== Calling Function ===
Calling function: multiply with args: {"a": 121, "b": 2}
Got output: 242

Once upon a time, in a cozy little hole in the wall, there lived a group of 242 mice. They were a lively and adventurous bunch, always eager to explore and discover new things. One day, they stumbled upon a hidden treasure trove filled with delicious cheese and tasty treats. The mice were overjoyed and quickly set to work, nibbling and nuzzling the treasure until it was all gone. From that day on, the group of 242 mice lived happily ever after, always remembering the magical day they discovered the treasure trove.

### Async Streaming Chat

In [ ]:
response = await agent.astream_chat(
    "What is 121 + 8? Once you have the answer, use that number to write a"
    " story about a group of mice."
)

response_gen = response.response_gen

async for token in response.async_response_gen():
    print(token, end="")

Added user message to memory: What is 121 + 8? Once you have the answer, use that number to write a story about a group of mice.
=== Calling Function ===
Calling function: add with args: {"a": 121, "b": 8}
Got output: 129

Once upon a time, in a cozy little hole in the wall, there lived a group of 129 mice. They were a lively and adventurous bunch, always eager to explore and discover new things. One day, they stumbled upon a hidden garden filled with beautiful flowers and delicious berries. The mice were overjoyed and quickly set to work, nibbling and nuzzling the flowers and berries until they were all gone. From that day on, the group of 129 mice lived happily ever after, always remembering the magical day they discovered the hidden garden.

### Agent with Personality

You can specify a system prompt to give the agent additional instruction or personality.

In [ ]:
from llama_index.core.prompts.system import SHAKESPEARE_WRITING_ASSISTANT

In [ ]:
agent = NVIDIAAgent.from_tools(
    [multiply_tool, add_tool],
    llm=NVIDIA("meta/llama-3.1-70b-instruct"),
    verbose=True,
    system_prompt=SHAKESPEARE_WRITING_ASSISTANT,
)

In [ ]:
response = agent.chat("Hi")
print(response)

Added user message to memory: Hi
Fair greeting unto thee! 'Tis a pleasure to converse with one such as thyself. How doth thy day fare? Doth thou seek inspiration for a tale, a poem, or perhaps a song, penned in the grand style of the Bard himself?


In [ ]:
response = agent.chat("Tell me a story")
print(response)

Added user message to memory: Tell me a story
Fair listener, gather 'round and heed my words, for I shall spin a yarn of love, of loss, and of longing, set amidst the rolling hills and verdant forests of a bygone era.
